### Best Practices
1. Preprocessing and Cleaning
2. Train test Split
3. BOW,TFIDF,Word2Vec
4. Train ML Algorithm

In [1]:
# load the dataset

import pandas as pd 
df = pd.read_csv('./all_kindle_review.csv')

In [2]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [3]:
df = df[['reviewText','rating']]

In [4]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [5]:
df.shape

(12000, 2)

In [6]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [7]:
df['rating'].unique()

array([3, 5, 4, 2, 1])

In [8]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

## Preprocessing and cleaning

In [9]:
# positive review s 1 and negative review is 0

df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)

In [10]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [11]:
df['rating'].unique()

array([1, 0])

In [12]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [13]:
### Lower all the cases

df['reviewText']=df['reviewText'].str.lower()

In [14]:

import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
%pip install lxml

import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_review(text):
    if not isinstance(text, str):
        return ""
    
    # Remove URLs
    text = re.sub(r'(http|https|ftp|ssh)://\S+', '', text)
    
    # Remove HTML tags
    text = BeautifulSoup(text, 'lxml').get_text()
    
    # Remove special characters but keep alphanumeric, space, and hyphen
    text = re.sub(r'[^a-zA-Z0-9\s-]', '', text)
    
    # Lowercase
    text = text.lower()
    
    # Remove stopwords
    text = " ".join([word for word in text.split() if word not in stop_words])
    
    # Remove extra spaces
    text = " ".join(text.split())
    
    return text

df['reviewText'] = df['reviewText'].apply(clean_review)


Note: you may need to restart the kernel to use updated packages.


In [16]:
## Lemmatizer

from nltk.stem import WordNetLemmatizer

In [17]:
lemmatizer = WordNetLemmatizer()

In [18]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [19]:
df['reviewText'] = df['reviewText'].apply(lambda x: lemmatize_words(x))

In [20]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [22]:

## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df['reviewText'],df['rating'],
                                              test_size=0.20)

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer()

x_train_bow = bow.fit_transform(X_train).toarray()
x_test = bow.transform(X_test).toarray()


In [24]:

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [25]:

from sklearn.naive_bayes import GaussianNB
nb_model_bow=GaussianNB().fit(x_train_bow,y_train)
nb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

In [26]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [27]:
y_pred_bow=nb_model_bow.predict(x_test)

In [28]:

y_pred_tfidf=nb_model_bow.predict(X_test_tfidf)

In [29]:

confusion_matrix(y_test,y_pred_bow)

array([[491, 254],
       [772, 883]])

In [30]:

print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))

BOW accuracy:  0.5725


In [31]:

confusion_matrix(y_test,y_pred_tfidf)

array([[481, 264],
       [762, 893]])

In [32]:

print("TFIDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))

TFIDF accuracy:  0.5725
